# Data Cleaning — Goodbooks-10k Top 100 (Updated)
**Objetivo:**  
 
1. Cargar y explorar los ficheros de ratings, metadatos y tags.  
2. Seleccionar los 100 libros más valorados.  
3. Filtrar usuarios con actividad mínima (≥ 20 ratings).  
4. Binarizar las calificaciones (≥ 4 → “like”).  
5. Pivotar a matriz dispersa 0/1 `R_binary`.  
6. Extraer y guardar metadata enriquecida (título, autor, top-tags).



Importamos las librerías

In [ ]:
import pandas as pd
import numpy as np
import pathlib as Path

: 

Cargamos los datos

In [ ]:
data_dir = Path("data")

ratings = pd.read_csv(data_dir / "ratings.csv")
to_read = pd.read_csv(data_dir / "to_read.csv")
books = pd.read_csv(data_dir / "books.csv")
book_tags = pd.read_csv(data_dir / "book_tags.csv")
tags = pd.read_csv(data_dir / "tags.csv")

print("Ratings shape:", ratings.shape)
print("Books shape:", books.shape)
print("Tags shape:", tags.shape)

: 